# Laba 1

In [1]:
import graphblas as gb
from graphblas import dtypes, monoid
from graphblas.io import mmread
from scipy.sparse import coo_matrix
import time
import numpy as np
import pandas as pd
import plotly.express as px

## #1 Наивный алгоритм

In [2]:
def count_triangles_naive(A):
  A2 = A.mxm(A)
  A3 = A.mxm(A2)
  diag = A3.diag()
  
  return int(diag.reduce() or 0)/6

## #2 Наивный алгоритм с маской

In [3]:
def count_triangles_mask(A):
  A2 = A.mxm(A)
  mask = A.dup(dtype=dtypes.BOOL)
  A2mask = A2.dup(mask=mask)
  sum_triangles = int(A2mask.reduce_scalar() or 0)
  
  return sum_triangles/6

## #3 Сohen's algorithm

In [4]:
def count_triangles_cohen(A):
    L = gb.select.tril(A)
    U = gb.select.triu(A)
    mask = A.dup(dtype=dtypes.BOOL)
    LUmask = L.mxm(U).dup(mask=mask)
    num_triangles_pair = int(LUmask.reduce_scalar() or 0)

    return num_triangles_pair / 2

## #4 Sandia algorithm

In [5]:
def count_triangles_sandia(A):
    L = gb.select.tril(A)
    mask = L.dup(dtype=dtypes.BOOL)
    L2mask = L.mxm(L).dup(mask=mask)
    num_triangles = int(L2mask.reduce_scalar() or 0)
    
    return num_triangles

## #5 Количество треугольников для каждой вершины

In [6]:
def count_triangles(A, return_type="np"):
    A2 = A.mxm(A)
    mask = A.dup(dtype=dtypes.BOOL)
    A2mask = A2.dup(mask=mask)
    
    triangles_vector = A2mask.reduce_columnwise(monoid.plus) / 2
    
    rows = A.shape[0]
    miss_index = set(range(rows)) - set(triangles_vector)
    
    for i in miss_index:
        triangles_vector[i] << 0.0
   
    return triangles_vector

## #6 Tecты

In [7]:
def test(A):
    start = time.time()
    print('Naive:', count_triangles_naive(A))
    end = time.time()
    print((end-start)*1000)

    start = time.time()
    print('Naive_mask:', count_triangles_mask(A))
    end = time.time()
    print((end-start)*1000)

    start = time.time()
    print('Cohen:', count_triangles_cohen(A))
    end = time.time()
    print((end-start)*1000)

    start = time.time()
    print('Sandia:', count_triangles_sandia(A))
    end = time.time()
    print((end-start)*1000)    

## #7 Проверка результатов

In [8]:
import os

files = os.listdir('graphs')
for file in files:
    M = mmread(f'graphs/{file}')    
    print(f'------------------ Файл: {file} ------------------')
    test(M)
   # print('__________________________________________')

------------------ Файл: crack.mtx ------------------
Naive: 20141.0
35.29524803161621
Naive_mask: 20141.0
0.0
Cohen: 20141.0
7.024049758911133
Sandia: 20141
5.106449127197266
------------------ Файл: email.mtx ------------------
Naive: 5343.0
26.27086639404297
Naive_mask: 5343.0
0.0
Cohen: 5343.0
0.0
Sandia: 5343
0.0
------------------ Файл: G1.mtx ------------------
Naive: 18093.0
126.01804733276367
Naive_mask: 18093.0
2.999544143676758
Cohen: 18093.0
1.7201900482177734
Sandia: 18093
0.0
------------------ Файл: G2.mtx ------------------
Naive: 18194.0
100.38185119628906
Naive_mask: 18194.0
0.0
Cohen: 18194.0
0.0
Sandia: 18194
0.0
------------------ Файл: G25.mtx ------------------
Naive: 1327.0
104.2635440826416
Naive_mask: 1327.0
0.0
Cohen: 1327.0
0.0
Sandia: 1327
12.54129409790039
------------------ Файл: G4.mtx ------------------
Naive: 18096.0
64.78261947631836
Naive_mask: 18096.0
0.0
Cohen: 18096.0
0.0
Sandia: 18096
0.0
------------------ Файл: G45.mtx ------------------
Naive:

In [9]:
# Проверка работы функции для подсчета кол-ва треугольников для каждой вершины
count_triangles(M)

gb.VectorExpression                         size  dtype
v_10.apply(binary.truediv[FP64], right=2)  11806   FP64

"Result"   nvals   size  dtype
gb.Vector  11806  11806   FP64
------------------------------
index   0     1     2     3     4     5     6     7     8     9      ...  \
value  1989.0   0.0   1.0   0.0   4.0   0.0   1.0   0.0   0.0   1.0  ...   

index 11796 11797 11798 11799 11800 11801 11802 11803 11804 11805  
value   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

Do expr.new() or other << expr to calculate the expression.

## #8 Генератор случайных графов

In [10]:
def generate_random_graph(n, density):
    matrix = np.zeros((n, n))

    for i in range(1, n+1):
        for j in range(i, n+1):
            if np.random.rand() < density:
                matrix[i-1, j-1] = 1

    np.fill_diagonal(matrix, 0)

    for i in range(1, n+1):
        for j in range(i, n+1):
            if matrix[i-1, j-1] == 1:
                matrix[j-1, i-1] = 1

    matrix_coo = coo_matrix(matrix)
    rows = matrix_coo.row
    cols = matrix_coo.col
    matrix = gb.Matrix.from_coo(
        rows,
        cols,
        np.ones(len(cols)),
        nrows=len(matrix_coo.diagonal()),
        ncols=len(matrix_coo.diagonal())
    )
    return matrix

In [11]:
generate_random_graph(10,0.1)

"M_122"    nvals  nrows  ncols  dtype     format
gb.Matrix      6     10     10   FP64  csr (iso)
------------------------------------------------
  0    1 2 3 4 5    6    7    8    9
0                                   
1                 1.0               
2                                   
3                                   
4                                   
5                                   
6    1.0                            
7                           1.0     
8                      1.0       1.0
9                           1.0

In [12]:
res1 = pd.DataFrame(columns = ['method', 'N', 'density', 'time'])
Ns = [100, 1000, 5000, 7000]
for N in Ns:
    density = 10/N**2
    M = generate_random_graph(N, density)
    
    start = time.time()
    count_triangles_naive(M)
    end = time.time()
    res1.loc[len(res1)] = ['Naive', N, density, (end-start)*1000]
    
    start = time.time()
    count_triangles_mask(M)
    end = time.time()
    res1.loc[len(res1)] = ['Naive_mask', N, density, (end-start)*1000]
    start = time.time()
    
    count_triangles_cohen(M)
    end = time.time()
    res1.loc[len(res1)] = ['Cohen', N, density, (end-start)*1000]
    start = time.time()
    
    count_triangles_sandia(M)
    end = time.time()
    res1.loc[len(res1)] = ['Sandia', N, density, (end-start)*1000]

In [13]:
res1

,method,N,density,time
0,Naive,100,1.000000e-03,3.999233
1,Naive_mask,100,1.000000e-03,1.000166
2,Cohen,100,1.000000e-03,0.998735
3,Sandia,100,1.000000e-03,0.000000
4,Naive,1000,1.000000e-05,0.000000
5,Naive_mask,1000,1.000000e-05,0.000000
6,Cohen,1000,1.000000e-05,1.431704
7,Sandia,1000,1.000000e-05,0.000000
8,Naive,5000,4.000000e-07,0.000000
9,Naive_mask,5000,4.000000e-07,0.000000


In [14]:
px.scatter(res1, x = 'N', y = 'time', color = 'method')

При очень разреженных графах Sandia и Cohen работают дольше

Наивный алгоритм и наивный алгоритм с маской -- наиболее удачные варианты для решения задачи на очень разреженных графах 

In [15]:
res2 = pd.DataFrame(columns = ['method', 'N', 'density', 'time'])
Density = [0.005, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
for density in Density:
    N = 500
    M = generate_random_graph(N, density)
    
    start = time.time()
    count_triangles_naive(M)
    end = time.time()
    res2.loc[len(res2)] = ['Naive', N, density, (end-start)*1000]
    
    start = time.time()
    count_triangles_mask(M)
    end = time.time()
    res2.loc[len(res2)] = ['Naive_mask', N, density, (end-start)*1000]
    start = time.time()
    
    count_triangles_cohen(M)
    end = time.time()
    res2.loc[len(res2)] = ['Cohen', N, density, (end-start)*1000]
    start = time.time()
    
    count_triangles_sandia(M)
    end = time.time()
    res2.loc[len(res2)] = ['Sandia', N, density, (end-start)*1000]

In [16]:
res2

,method,N,density,time
0,Naive,500,0.005,1.179457
1,Naive_mask,500,0.005,0.000000
2,Cohen,500,0.005,0.000000
3,Sandia,500,0.005,0.000000
4,Naive,500,0.010,20.248413
5,Naive_mask,500,0.010,0.999212
6,Cohen,500,0.010,7.255077
7,Sandia,500,0.010,1.003504
8,Naive,500,0.100,15.651464
9,Naive_mask,500,0.100,0.000000


In [17]:
px.scatter(res2, x = 'density', y = 'time', color = 'method')

С увеличением плотности при неизменном размере графа время увеличивается

Наивный алгоритм самый долгий

Cohen и Sandia наиболее быстрые